In [2]:
import os
import sys
from open_clip import create_model_and_transforms
from open_clip import tokenize 
import torch
import numpy as np
import pandas as pd
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import shutil
import torch.nn.functional as F


/root/miniconda3/envs/dac/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
validation_data = pd.read_csv("../CC_10000/val_1000.csv")

In [4]:
validation_data.head()

,url,image_file_name,quality_captions,LLM_dense_captions
0,http://l7.alamy.com/zooms/400a6896682a4578b028...,1435576_3694708615.jpg,quality_captions/1435576_3694708615.json,LLM_dense/1435576_3694708615.json
1,http://l7.alamy.com/zooms/889834cf95fd4cf58c7e...,1436456_3127317779.jpg,quality_captions/1436456_3127317779.json,LLM_dense/1436456_3127317779.json
2,http://img-aws.ehowcdn.com/560x560p/photos.dem...,851376_2084137130.jpg,quality_captions/851376_2084137130.json,LLM_dense/851376_2084137130.json
3,https://image.shutterstock.com/z/stock-photo-c...,1437128_1476564219.jpg,quality_captions/1437128_1476564219.json,LLM_dense/1437128_1476564219.json
4,https://ak3.picdn.net/shutterstock/videos/5612...,1437550_71712799.jpg,quality_captions/1437550_71712799.json,LLM_dense/1437550_71712799.json


In [8]:
model, _, image_preprocess = create_model_and_transforms(
        "ViT-B/32",
        "openai",
        precision="amp",
        device="cpu",
        jit=False,
        force_quick_gelu=False,
        pretrained_image=False,
        image_mean=None,
        image_std=None,
        lora=4,
        freeze_img=False,
        kqv_lora=False,
    )
model.to("cuda")

Model loaded
Missing Keys: []
Unexpected Keys: []


CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (ln_attn): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (ln): Identity()
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
        )
        (1): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
    

In [20]:
checkpoint = torch.load("logs/baseline/checkpoints/epoch_10.pt", map_location="cuda:0")  ### replace your model weight here.
sd = checkpoint["state_dict"]
if next(iter(sd.items()))[0].startswith("module"):
    sd = {k[len("module.") :]: v for k, v in sd.items()}
model.load_state_dict(sd)
model.to("cuda")
model.eval()

CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (ln_attn): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (ln): Identity()
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
        )
        (1): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
    

In [18]:
def get_retrieval_scores_with_loss_batch(model, data, device="cuda"):
    # Preprocessing for images
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Adjust size to match model requirements
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    model.eval()  # Set the model to evaluation mode
    cumulative_loss = 0.0
    num_samples = 0
    all_logits_per_image = []
    all_logits_per_text = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for i in range(0, len(data), 32):  # Batch size of 4 (can be adjusted)
            batch_data = data.iloc[i:i+32]
            images = []
            captions = []

            # Load and preprocess images and captions for the batch
            for _, row in batch_data.iterrows():
                # Load and preprocess image
                image_path = "../CC_10000/validation/" + row.image_file_name
                try:
                    image = Image.open(image_path).convert("RGB")
                except Exception as e:
                    print("image corrupted")
                    dummy_array = np.full((224, 224, 3), 128, dtype=np.uint8)  # Gray image
                    image = Image.fromarray(dummy_array)
                images.append(preprocess(image))

                # Load caption text
                caption_path = "../CC_10000/"+row.quality_captions	
                with open(caption_path, "r") as f:
                    captions.append(f.read().strip())

            # Convert images to tensor and move to device
            images = torch.stack(images).to(device)

            # Tokenize captions and move to device
            tokenized_captions = tokenize(captions).to(device)

            # Compute embeddings for all images and texts
            image_embeddings = model(images, None)
            text_embeddings = model(None, tokenized_captions)

            # Normalize embeddings
            image_embeddings = image_embeddings / torch.norm(image_embeddings, dim=1, keepdim=True)
            text_embeddings = text_embeddings / torch.norm(text_embeddings, dim=1, keepdim=True)

            # Compute similarity logits
            logits_per_image = torch.matmul(image_embeddings, text_embeddings.T)  # Image-to-text similarities
            logits_per_text = logits_per_image.T  # Text-to-image similarities

            # Store logits for potential later use
            all_logits_per_image.append(logits_per_image)
            all_logits_per_text.append(logits_per_text)

            # Compute cross-entropy loss
            labels = torch.arange(logits_per_image.shape[0], device=device)  # Ground truth indices
            loss_per_image = F.cross_entropy(logits_per_image, labels)
            loss_per_text = F.cross_entropy(logits_per_text, labels)
            total_loss = (loss_per_image + loss_per_text) / 2

            # Accumulate loss
            cumulative_loss += total_loss.item() * logits_per_image.shape[0]
            num_samples += logits_per_image.shape[0]

    # Compute average loss
    avg_loss = cumulative_loss / num_samples

    # Return metrics
    metrics = {
        "avg_loss": avg_loss,
    }

    return metrics

In [ ]:
get_retrieval_scores_with_loss_batch(model, validation_data, device="cuda")